In [1]:
import pandas as pd
import tensorflow as tf
from scikeras.wrappers import KerasClassifier 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix, roc_curve, precision_recall_curve, auc
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Read initial data
file_path='rtfDataSet.csv'
df = pd.read_csv(file_path)
print(df.head())

       0      1      2      3      4      5      6      7      8      9  ...  \
0 -0.536  0.292 -0.684  0.123 -0.118  0.346 -0.308 -0.113  0.401 -0.399  ...   
1 -0.496 -0.298 -0.529  0.073 -0.072  0.107 -0.316 -0.066 -0.168 -0.099  ...   
2  0.008 -0.031 -0.240  0.178 -0.120  0.317 -0.481  0.031 -0.077  0.063  ...   
3 -0.188 -0.180 -0.062 -0.104 -0.136 -0.061 -0.216 -0.143  0.068 -0.189  ...   
4  0.038  0.155 -0.203 -0.088 -0.084 -0.164 -0.145 -0.168  0.008  0.143  ...   

      61     62     63     64     65     66     67     68     69  label  
0 -0.135 -0.063 -0.410 -0.223 -0.599 -0.136 -0.329 -0.132 -0.266      0  
1  0.189  0.007 -0.362 -0.151 -0.338 -0.031 -0.159 -0.097 -0.131      0  
2 -0.035  0.101 -0.098 -0.172 -0.290  0.033 -0.342 -0.321 -0.238      0  
3  0.146 -0.145 -0.527 -0.292 -0.663 -0.078 -0.194 -0.151 -0.268      0  
4  0.019 -0.084  0.164 -0.165 -0.126 -0.112  0.029 -0.138  0.148      1  

[5 rows x 71 columns]


In [3]:
#Reduce to sets and normalized
set_2=[2, 5, 9, 10, 16, 21, 28, 34, 38, 48, 49, 55, 61, 62, 64]
set_3=[1, 2, 7, 8, 9, 10, 19, 23, 28, 34, 35, 36, 42, 44, 45, 47, 48, 49, 53, 55, 56, 57, 58, 63, 64, 65, 67]

reduced_df_2=df.iloc[:,set_2]
reduced_df_3=df.iloc[:,set_3]

y = df.iloc[:, -1]

scaler = MinMaxScaler()

reduced_df_2 = scaler.fit_transform(reduced_df_2)
reduced_df_3 = scaler.fit_transform(reduced_df_3)

#Array of sets
reduced_array=[reduced_df_2,reduced_df_3]

In [13]:
def create_model(neurons, activation):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(neurons, activation=activation, input_dim=len(X)))
    model.add(tf.keras.layers.Dense(neurons, activation=activation))
    model.add(tf.keras.layers.Dense(neurons, activation=activation))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Grid search parameters
neurons = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
activations = ['relu', 'softmax', 'sigmoid', 'linear'] 

param_grid = dict(neurons=neurons, activation=activations)

# Perform grid search
for i in range(2):
    # Split data
    X = reduced_array[i]
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    skf.get_n_splits(X, y)
    # Create model
    model = KerasClassifier(build_fn=create_model(1,'relu'), verbose=0)
    # Grid search
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=skf, scoring='accuracy', error_score=0)
    grid_result = grid.fit(X, y)
    # Results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tmphygktbwe\assets


INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tmphygktbwe\assets


INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tmp29e721vm\assets


INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tmp29e721vm\assets


INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tmpkl1jtel_\assets


...

INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tmpojwe2wwn\assets


INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tmpojwe2wwn\assets


ValueError: Invalid parameter activation for estimator KerasClassifier.
This issue can likely be resolved by setting this parameter in the KerasClassifier constructor:
`KerasClassifier(activation=relu)`
Check the list of available parameters with `estimator.get_params().keys()`

INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tmpsck0ydgd\assets


INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tmpsck0ydgd\assets


## Model 1

In [10]:
#Creacion dinamica de folds y clasificacion y metricas
stratified_kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
counter=1
epochs = [1,50,100,150,200,250,300,350,400,450,500]

#Neural Network
for X in reduced_array:
    print("Set: ",counter)
    array_accuracy= []
    array_precision= []
    array_recall= []
    array_auc = []

    for fold,(train_index,val_index) in enumerate(stratified_kfold.split(X,y)):

        for epoch in epochs:
            X_train,X_val=X[train_index],X[val_index]
            y_train,y_val=y[train_index],y[val_index]

            tf.keras.backend.clear_session() # Clear session before building the model

            model = tf.keras.models.Sequential([
                tf.keras.layers.Dense(len(X), activation='relu', input_shape=(X_train.shape[1],)),
                tf.keras.layers.Dense(6, activation='relu'),
                tf.keras.layers.Dense(6, activation='relu'),
                tf.keras.layers.Dense(1, activation='softmax')
            ])

            optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)
            model.compile(optimizer=optimizer,
                        loss='binary_crossentropy',
                        metrics=['accuracy'],
                        )  
            
            model.fit(X_train, y_train, epochs=epoch, batch_size=32, validation_data=(X_val, y_val))


Set:  1
12/12 [==============================] - 1s 24ms/step - loss: 6.2233 - accuracy: 0.5112 - val_loss: 0.6925 - val_accuracy: 0.5000
Epoch 1/50
12/12 [==============================] - 1s 24ms/step - loss: 3.9576 - accuracy: 0.5112 - val_loss: 0.6919 - val_accuracy: 0.5000
Epoch 2/50
12/12 [==============================] - 0s 8ms/step - loss: 0.7079 - accuracy: 0.5112 - val_loss: 0.7086 - val_accuracy: 0.5000
Epoch 3/50
12/12 [==============================] - 0s 7ms/step - loss: 0.7045 - accuracy: 0.5112 - val_loss: 0.6999 - val_accuracy: 0.5000
Epoch 4/50
12/12 [==============================] - 0s 6ms/step - loss: 0.7053 - accuracy: 0.5112 - val_loss: 0.6951 - val_accuracy: 0.5000
Epoch 5/50
12/12 [==============================] - 0s 6ms/step - loss: 0.6989 - accuracy: 0.5112 - val_loss: 0.7005 - val_accuracy: 0.5000
Epoch 6/50
12/12 [==============================] - 0s 6ms/step - loss: 0.6964 - accuracy: 0.5112 - val_loss: 0.6934 - val_accuracy: 0.5000
Epoch 7/50
12/12 [===

KeyboardInterrupt: 

In [ ]:
#Neural Network
for X in reduced_array:
    print("Set: ",counter)
    array_accuracy= []
    array_precision= []
    array_recall= []
    array_auc = []

    for fold,(train_index,val_index) in enumerate(stratified_kfold.split(X,y)):

        for epoch in epochs:
            X_train,X_val=X[train_index],X[val_index]
            y_train,y_val=y[train_index],y[val_index]

            tf.keras.backend.clear_session() # Clear session before building the model

            model = tf.keras.models.Sequential([
                tf.keras.layers.Dense(6, activation='relu', input_shape=(X_train.shape[1],)),
                tf.keras.layers.Dense(6, activation='relu'),
                tf.keras.layers.Dense(6, activation='relu'),
                tf.keras.layers.Dense(1, activation='softmax')
            ])

            optimizer = tf.keras.optimizers.Adam(learning_rate=0.3)
            model.compile(optimizer=optimizer,
                        loss='binary_crossentropy',
                        metrics=['accuracy'],
                        )  
            
            model.fit(X_train, y_train, epochs=epoch, batch_size=32, validation_data=(X_val, y_val))

In [ ]:
#Neural Network
for X in reduced_array:
    print("Set: ",counter)
    array_accuracy= []
    array_precision= []
    array_recall= []
    array_auc = []

    for fold,(train_index,val_index) in enumerate(stratified_kfold.split(X,y)):

        for epoch in epochs:
            X_train,X_val=X[train_index],X[val_index]
            y_train,y_val=y[train_index],y[val_index]

            tf.keras.backend.clear_session() # Clear session before building the model

            model = tf.keras.models.Sequential([
                tf.keras.layers.Dense(6, activation='relu', input_shape=(X_train.shape[1],)),
                tf.keras.layers.Dense(6, activation='relu'),
                tf.keras.layers.Dense(6, activation='relu'),
                tf.keras.layers.Dense(1, activation='softmax')
            ])

            optimizer = tf.keras.optimizers.Adam(learning_rate=0.5)
            model.compile(optimizer=optimizer,
                        loss='binary_crossentropy',
                        metrics=['accuracy'],
                        )  
            
            model.fit(X_train, y_train, epochs=epoch, batch_size=32, validation_data=(X_val, y_val))

## Model 2

In [ ]:
#Neural Network
for X in reduced_array:
    print("Set: ",counter)
    array_accuracy= []
    array_precision= []
    array_recall= []
    array_auc = []

    for fold,(train_index,val_index) in enumerate(stratified_kfold.split(X,y)):

        for epoch in epochs:
            X_train,X_val=X[train_index],X[val_index]
            y_train,y_val=y[train_index],y[val_index]

            tf.keras.backend.clear_session() # Clear session before building the model

            model = tf.keras.models.Sequential([
                tf.keras.layers.Dense(8, activation='relu', input_shape=(X_train.shape[1],)),
                tf.keras.layers.Dense(8, activation='relu'),
                tf.keras.layers.Dense(8, activation='relu'),
                tf.keras.layers.Dense(1, activation='softmax')
            ])

            optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)
            model.compile(optimizer=optimizer,
                        loss='binary_crossentropy',
                        metrics=['accuracy'],
                        )  
            
            model.fit(X_train, y_train, epochs=epoch, batch_size=32, validation_data=(X_val, y_val))

In [ ]:
#Neural Network
for X in reduced_array:
    print("Set: ",counter)
    array_accuracy= []
    array_precision= []
    array_recall= []
    array_auc = []

    for fold,(train_index,val_index) in enumerate(stratified_kfold.split(X,y)):

        for epoch in epochs:
            X_train,X_val=X[train_index],X[val_index]
            y_train,y_val=y[train_index],y[val_index]

            tf.keras.backend.clear_session() # Clear session before building the model

            model = tf.keras.models.Sequential([
                tf.keras.layers.Dense(8, activation='relu', input_shape=(X_train.shape[1],)),
                tf.keras.layers.Dense(8, activation='relu'),
                tf.keras.layers.Dense(8, activation='relu'),
                tf.keras.layers.Dense(1, activation='softmax')
            ])

            optimizer = tf.keras.optimizers.Adam(learning_rate=0.3)
            model.compile(optimizer=optimizer,
                        loss='binary_crossentropy',
                        metrics=['accuracy'],
                        )  
            
            model.fit(X_train, y_train, epochs=epoch, batch_size=32, validation_data=(X_val, y_val))

In [ ]:
#Neural Network
for X in reduced_array:
    print("Set: ",counter)
    array_accuracy= []
    array_precision= []
    array_recall= []
    array_auc = []

    for fold,(train_index,val_index) in enumerate(stratified_kfold.split(X,y)):

        for epoch in epochs:
            X_train,X_val=X[train_index],X[val_index]
            y_train,y_val=y[train_index],y[val_index]

            tf.keras.backend.clear_session() # Clear session before building the model

            model = tf.keras.models.Sequential([
                tf.keras.layers.Dense(8, activation='relu', input_shape=(X_train.shape[1],)),
                tf.keras.layers.Dense(8, activation='relu'),
                tf.keras.layers.Dense(8, activation='relu'),
                tf.keras.layers.Dense(1, activation='softmax')
            ])

            optimizer = tf.keras.optimizers.Adam(learning_rate=0.5)
            model.compile(optimizer=optimizer,
                        loss='binary_crossentropy',
                        metrics=['accuracy'],
                        )  
            
            model.fit(X_train, y_train, epochs=epoch, batch_size=32, validation_data=(X_val, y_val))

## Model 3